<a href="https://colab.research.google.com/github/beingboy35/InnoByte-Internship/blob/main/InooByte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 0 — Install required libs (run once)
!pip install -q gdown plotly scikit-learn python-pptx openpyxl


In [ ]:
# Step 1 — Imports and display settings
import os, glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)
pd.set_option('display.max_columns', 120)


In [ ]:
# Step 2 — Get the dataset (two options)

# OPTION A — Download from Google Drive link in the project brief (automatic).
# File id from the brief: 1YrjYKtS1WHmINL6eafRsrDzrZaw2_WvX
file_id = "1YrjYKtS1WHmINL6eafRsrDzrZaw2_WvX"
# gdown will save with original filename when not using -O
!gdown --id $file_id || echo "gdown failed; try OPTION B (mount Drive or upload manually)"

# List likely data files in CWD
candidates = [f for f in os.listdir('.') if f.lower().endswith(('.csv','.xlsx','.xls','.zip','.json'))]
print("Detected files:", candidates)


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1YrjYKtS1WHmINL6eafRsrDzrZaw2_WvX
To: /content/Amazon Sale Report.csv
100% 19.1M/19.1M [00:00<00:00, 46.1MB/s]
Detected files: ['Amazon Sale Report.csv', 'Amazon Sale Report (1).csv']


In [ ]:
# If nothing was detected in OPTION A, use OPTION B - mount Google Drive and provide path:
# from google.colab import drive
# drive.mount('/content/drive')
# Then set path_to_file = "/content/drive/MyDrive/your-folder/yourfile.csv"

# We'll now attempt to load the first candidate file found automatically.
import zipfile
data_file = None
if candidates:
    data_file = candidates[0]
    print("Loading:", data_file)
    if data_file.lower().endswith('.csv') :
        df = pd.read_csv(data_file, low_memory=False)
    elif data_file.lower().endswith(('.xls','.xlsx')):
        df = pd.read_excel(data_file)
    elif data_file.lower().endswith('.json'):
        df = pd.read_json(data_file)
    elif data_file.lower().endswith('.zip'):
        with zipfile.ZipFile(data_file) as z:
            z.extractall('data_extracted')
        inner = glob.glob('data_extracted/*')
        # try to load the first csv/xlsx inside
        fp = next((x for x in inner if x.lower().endswith(('.csv','.xls','.xlsx'))), inner[0])
        if fp.lower().endswith('.csv'): df = pd.read_csv(fp, low_memory=False)
        else: df = pd.read_excel(fp)
    else:
        raise ValueError("Unsupported file type - open manually.")
else:
    raise FileNotFoundError("No dataset found. Upload the dataset to Colab or mount Drive and update path.")
print("Loaded dataframe with shape:", df.shape)
df.head().fillna('').iloc[:5]



Loading: Amazon Sale Report.csv
Loaded dataframe with shape: (128976, 21)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Category,Size,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B,fulfilled-by,New,PendingS
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,T-shirt,S,On the Way,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,False,Easy Ship,,
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,Shirt,3XL,Shipped,1,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,False,Easy Ship,,
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,Shirt,XL,Shipped,1,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,True,,,
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,Blazzer,L,On the Way,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,False,Easy Ship,,
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,Trousers,3XL,Shipped,1,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,False,,,


In [ ]:
# Step 3 — Quick EDA: structure, missing values, duplicates
print("Columns:", list(df.columns))
print("\nShape:", df.shape)
display(df.info())
display(df.describe(include='all').T.head(30))
print("\nTop missing columns:")
display(df.isnull().sum().sort_values(ascending=False).head(20))

# Basic duplicates check (by full row)
dups = df.duplicated().sum()
print(f"Full-row duplicates: {dups}")


Columns: ['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel', 'ship-service-level', 'Category', 'Size', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'B2B', 'fulfilled-by', 'New', 'PendingS']

Shape: (128976, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128976 entries, 0 to 128975
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128976 non-null  int64  
 1   Order ID            128976 non-null  object 
 2   Date                128976 non-null  object 
 3   Status              128976 non-null  object 
 4   Fulfilment          128976 non-null  object 
 5   Sales Channel       128976 non-null  object 
 6   ship-service-level  128976 non-null  object 
 7   Category            128976 non-null  object 
 8   Size                128976 non-null  object 
 9   Courier Status      128976 non-null  ob

None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
index,128976.0,NaN,NaN,NaN,64486.130427,37232.897832,0.0,32242.75,64486.5,96730.25,128974.0
Order ID,128976,120229,403-4984515-8861958,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date,128976,91,05-03-2022,2085,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Status,128976,13,Shipped,77815,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fulfilment,128976,2,Amazon,89713,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sales Channel,128976,2,Amazon.in,128852,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ship-service-level,128976,2,Expedited,88630,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category,128976,9,T-shirt,50292,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Size,128976,11,M,22373,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Courier Status,128976,4,Shipped,109486,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Top missing columns:


,0
PendingS,128976
New,128976
fulfilled-by,89713
currency,7800
Amount,7800
ship-state,35
ship-city,35
ship-postal-code,35
ship-country,35
index,0


Full-row duplicates: 168


In [ ]:
# Step 4 — Standardize column names & auto-detect important fields
df.columns = [c.strip().lower().replace(' ','_').replace('-','_') for c in df.columns]

# Candidate detectors
def first_match(cols, keywords):
    for k in keywords:
        for c in cols:
            if k in c:
                return c
    return None

cols = df.columns.tolist()
order_id_col = first_match(cols, ['order_id','orderid','order'])
cust_col     = first_match(cols, ['customer_id','customer','buyer','buyer_id','cust_id','email'])
date_col     = first_match(cols, ['order_date','date','created_at','created'])
amount_col   = first_match(cols, ['amount','total','order_value','order_total','price','revenue'])
qty_col      = first_match(cols, ['quantity','qty','units','item_qty'])
category_col = first_match(cols, ['category','product_category','product_type','item_category'])
product_col  = first_match(cols, ['product','sku','item_name','title','product_name'])
fulfill_col  = first_match(cols, ['fulfil','fulfill','shipment','shipping_method','fulfillment'])
state_col    = first_match(cols, ['state','region','province'])
city_col     = first_match(cols, ['city','town'])
lat_col      = first_match(cols, ['latitude','lat'])
lon_col      = first_match(cols, ['longitude','lon','lng'])

print("Auto-detected columns:")
print("order_id:", order_id_col)
print("customer_id:", cust_col)
print("date:", date_col)
print("amount:", amount_col)
print("quantity:", qty_col)
print("category:", category_col)
print("product:", product_col)
print("fulfillment:", fulfill_col)
print("state:", state_col)
print("city:", city_col)
print("lat/lon:", lat_col, lon_col)

# If any are None, you can manually set them, e.g.
# date_col = 'your_date_column_name'


Auto-detected columns:
order_id: order_id
customer_id: None
date: date
amount: amount
quantity: qty
category: category
product: None
fulfillment: fulfilment
state: ship_state
city: ship_city
lat/lon: None None


In [ ]:
# Step 5 — Parse dates and numeric cleaning (robust)
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce', dayfirst=False)
    df = df.sort_values(by=[date_col])
    df.rename(columns={date_col:'order_date'}, inplace=True)
else:
    print("No date column detected automatically. If your date column is named differently, set date_col variable.")

if amount_col:
    df[amount_col] = df[amount_col].astype(str).str.replace('[,$]','', regex=True).str.replace('₹','', regex=True)
    df[amount_col] = pd.to_numeric(df[amount_col].str.replace(' ','', regex=False), errors='coerce')
    df.rename(columns={amount_col:'order_amount'}, inplace=True)
else:
    print("No amount-like column detected. Set amount_col manually if exists.")

if qty_col:
    df.rename(columns={qty_col:'quantity'}, inplace=True)
else:
    # create a default quantity if not found
    df['quantity'] = 1

# Trim strings
for c in df.select_dtypes(include='object').columns:
    df[c] = df[c].astype(str).str.strip()

print("After cleaning, shape:", df.shape)
df[['order_date','order_amount','quantity']].head().fillna('')


/tmp/ipython-input-4055170776.py:3: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



After cleaning, shape: (128976, 21)


,order_date,order_amount,quantity
48971,2022-03-31,495.00,1
49002,2022-03-31,599.00,1
49003,2022-03-31,416.19,0
49004,2022-03-31,449.00,1
49005,2022-03-31,542.00,1


In [ ]:
# Step 6 — Feature engineering (time features)
if 'order_date' in df.columns:
    df['year'] = df['order_date'].dt.year
    df['month'] = df['order_date'].dt.to_period('M').astype(str)
    df['week'] = df['order_date'].dt.isocalendar().week
    df['weekday'] = df['order_date'].dt.day_name()
else:
    print("No order_date to derive time features from.")


In [ ]:
# Step 7 — Sales overview KPIs
sales_col = 'order_amount' if 'order_amount' in df.columns else amount_col
orders_count = df[order_id_col].nunique() if order_id_col and order_id_col in df.columns else df.shape[0]
total_sales = df[sales_col].sum() if sales_col in df.columns else np.nan
avg_order_value = total_sales / orders_count if orders_count else np.nan
total_units = df['quantity'].sum()

print(f"Total sales: {total_sales:,.2f}")
print(f"Orders count: {orders_count}")
print(f"Avg order value: {avg_order_value:,.2f}")
print(f"Units sold: {total_units}")


Total sales: 78,590,170.25
Orders count: 120229
Avg order value: 653.67
Units sold: 116646


In [ ]:
# Time series: daily and monthly sales trend (interactive)
if 'order_date' in df.columns and sales_col in df.columns:
    daily = df.groupby(pd.Grouper(key='order_date', freq='D'))[sales_col].sum().reset_index()
    fig = px.line(daily, x='order_date', y=sales_col, title='Daily Sales')
    fig.show()

    monthly = df.groupby('month')[sales_col].sum().reset_index().sort_values('month')
    fig2 = px.bar(monthly, x='month', y=sales_col, title='Monthly Sales')
    fig2.update_xaxes(type='category')
    fig2.show()
else:
    print("Can't draw time series — missing order_date or sales column.")


In [ ]:
# Step 8 — Product / Category analysis: top categories & top products by revenue and volume
cat = category_col if category_col in df.columns else (category_col or None)
prod = product_col if product_col in df.columns else (product_col or None)

if cat:
    by_cat = df.groupby(cat).agg(revenue=(sales_col,'sum'), units=('quantity','sum')).reset_index().sort_values('revenue', ascending=False)
    display(by_cat.head(10))
    fig = px.bar(by_cat.head(10), x=cat, y='revenue', title='Top 10 Categories by Revenue')
    fig.show()
else:
    print("Category column not found automatically; set category_col to your column name.")

if prod:
    by_prod = df.groupby(prod).agg(revenue=(sales_col,'sum'), units=('quantity','sum')).reset_index().sort_values('revenue', ascending=False)
    display(by_prod.head(15))
    fig = px.bar(by_prod.head(12), x=prod, y='revenue', title='Top 12 Products by Revenue')
    fig.show()
else:
    print("Product column not found automatically; set product_col to your column name.")


,category,revenue,units
5,T-shirt,39206756.65,45292
2,Shirt,21297770.08,45044
0,Blazzer,11215104.12,13943
6,Trousers,5346286.30,9898
1,Perfume,789419.66,1051
7,Wallet,458408.18,863
4,Socks,150757.50,399
3,Shoes,124752.76,153
8,Watch,915.00,3


Product column not found automatically; set product_col to your column name.


In [ ]:
# Step 9 — Fulfillment analysis (which fulfillment methods perform best)
ful = fulfill_col if fulfill_col in df.columns else (fulfill_col or None)
if ful and ful in df.columns:
    f = df.groupby(ful).agg(orders=(order_id_col,'nunique' if order_id_col in df.columns else 'count'), revenue=(sales_col,'sum')).reset_index().sort_values('revenue', ascending=False)
    display(f)
    fig = px.bar(f, x=ful, y='revenue', title='Revenue by Fulfillment Method')
    fig.show()
else:
    print("Fulfillment method not found automatically.")


,fulfilment,orders,revenue
0,Amazon,83906,54327540.00
1,Merchant,36323,24262630.25


In [ ]:
# Step 10 — Geographical analysis: top states / cities (bar charts). Map if lat/lon present.
geo = state_col if state_col in df.columns else state_col
city = city_col if city_col in df.columns else city_col

if geo and geo in df.columns:
    by_state = df.groupby(geo).agg(revenue=(sales_col,'sum'), orders=(order_id_col,'nunique' if order_id_col in df.columns else 'count')).reset_index().sort_values('revenue', ascending=False)
    display(by_state.head(15))
    fig = px.bar(by_state.head(15), x=geo, y='revenue', title='Top States by Revenue')
    fig.show()
elif city and city in df.columns:
    by_city = df.groupby(city).agg(revenue=(sales_col,'sum')).reset_index().sort_values('revenue', ascending=False)
    display(by_city.head(15))
    fig = px.bar(by_city.head(15), x=city, y='revenue', title='Top Cities by Revenue')
    fig.show()
else:
    print("No geographic columns detected automatically. If you have 'state' or 'city', set state_col/city_col.")


,ship_state,revenue,orders
28,MAHARASHTRA,13340333.05,20756
23,KARNATAKA,10480694.22,16162
57,TELANGANA,6915018.08,10394
59,UTTAR PRADESH,6823947.08,10057
56,TAMIL NADU,6519182.30,10507
14,DELHI,4232738.97,6347
24,KERALA,3823559.58,6097
61,WEST BENGAL,3507212.82,5645
1,ANDHRA PRADESH,3217859.86,4970
19,HARYANA,2880355.99,4134


In [ ]:
# If lat/lon present (scatter)
if lat_col and lon_col and lat_col in df.columns and lon_col in df.columns and sales_col in df.columns:
    fig = px.scatter_geo(df, lat=lat_col, lon=lon_col, size=sales_col, hover_name=city if city in df.columns else None,
                         title='Sales by Location (bubble size=order amount)')
    fig.update_layout(geo_scope='world')
    fig.show()


In [ ]:
# Step 11 — Customer segmentation (RFM + KMeans)
# Requires a customer identifier
cust = cust_col if cust_col in df.columns else cust_col
if cust and cust in df.columns and 'order_date' in df.columns and sales_col in df.columns:
    # RFM
    snapshot_date = df['order_date'].max() + timedelta(days=1)
    rfm = df.groupby(cust).agg({
        'order_date': lambda x: (snapshot_date - x.max()).days,
        order_id_col if order_id_col in df.columns else 'order_date': ('nunique' if order_id_col in df.columns else 'count'),
        sales_col: 'sum'
    }).reset_index()
    rfm.columns = [cust, 'recency', 'frequency', 'monetary']
    rfm = rfm.fillna(0)
    # Scores
    rfm['r_score'] = pd.qcut(rfm['recency'], 4, labels=[4,3,2,1]).astype(int)
    rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 4, labels=[1,2,3,4]).astype(int)
    rfm['m_score'] = pd.qcut(rfm['monetary'], 4, labels=[1,2,3,4]).astype(int)
    rfm['rfm_score'] = rfm['r_score'].map(str)+rfm['f_score'].map(str)+rfm['m_score'].map(str)
    display(rfm.head())

    # KMeans on scaled RFM
    X = rfm[['recency','frequency','monetary']].values
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    inertias = []
    for k in range(1,7):
        inertias.append(KMeans(n_clusters=k, random_state=42).fit(Xs).inertia_)
    plt.plot(range(1,7), inertias, marker='o')
    plt.xlabel('k'); plt.ylabel('inertia'); plt.title('Elbow method for K')
    plt.show()

    # choose k (e.g., k=3 or 4). We'll pick 4 as a reasonable default.
    k = 4
    kmeans = KMeans(n_clusters=k, random_state=42).fit(Xs)
    rfm['cluster'] = kmeans.labels_
    display(rfm.groupby('cluster').agg({
        'recency':'median','frequency':'median','monetary':'median','rfm_score':'count'
    }).rename(columns={'rfm_score':'count'}))
else:
    print("Customer column or necessary columns not found. RFM requires customer_id, order_date & amount.")


Customer column or necessary columns not found. RFM requires customer_id, order_date & amount.


In [ ]:
# Step 12 — Cohort analysis (monthly cohorts)
if cust and cust in df.columns and 'order_date' in df.columns:
    df['order_month'] = df['order_date'].dt.to_period('M').astype(str)
    df['cohort_month'] = df.groupby(cust)['order_date'].transform('min').dt.to_period('M').astype(str)
    cohort = df.groupby(['cohort_month','order_month']).agg(n_customers=(cust,'nunique')).reset_index()
    cohort_pivot = cohort.pivot_table(index='cohort_month', columns='order_month', values='n_customers')
    # percent retained
    cohort_size = cohort_pivot.iloc[:,0]
    retention = cohort_pivot.divide(cohort_size, axis=0)
    display(cohort_pivot.head())
    plt.figure(figsize=(12,6))
    plt.title('Cohort table (raw counts)')
    sns_matrix = retention.fillna(0)
    import seaborn as sns
    sns.heatmap(sns_matrix, annot=True, fmt=".2f", cmap='Blues')
    plt.show()
else:
    print("Skipping cohort analysis — missing customer or order_date.")


Skipping cohort analysis — missing customer or order_date.


In [ ]:
# Step 13 — Save cleaned datasets & export visuals
os.makedirs('results', exist_ok=True)
df.to_csv('results/cleaned_amazon_sales.csv', index=False)
print("Saved cleaned data to results/cleaned_amazon_sales.csv")
if 'rfm' in globals():
    rfm.to_csv('results/customer_rfm.csv', index=False)
    print("Saved RFM results to results/customer_rfm.csv")


Saved cleaned data to results/cleaned_amazon_sales.csv


In [ ]:
# Optional Step 14 — Quick insights dump (print top 5 actionables)
print("QUICK ACTIONABLES (auto-generated):")
try:
    top_cats = by_cat.head(3)[cat].tolist() if cat else []
    top_states = by_state.head(3)[geo].tolist() if geo and 'by_state' in globals() else []
    print(f"- Focus marketing on top categories: {top_cats}")
    print(f"- Stock top SKUs from top products list (see Top Products output).")
    if 'rfm' in globals():
        print("- Run retention campaigns for high-value clusters (see RFM clustering).")
    if ful and ful in df.columns:
        print("- Re-evaluate slower/low-performance fulfillment methods (see Fulfillment output).")
    print("- Create monthly replenishment plan from top-selling SKUs & states.")
except Exception as e:
    print("Could not generate auto-insights for all items. Inspect outputs above.")


QUICK ACTIONABLES (auto-generated):
- Focus marketing on top categories: ['T-shirt', 'Shirt', 'Blazzer']
- Stock top SKUs from top products list (see Top Products output).
- Re-evaluate slower/low-performance fulfillment methods (see Fulfillment output).
- Create monthly replenishment plan from top-selling SKUs & states.
